**Install Dependencies**

In [ ]:
!pip install groq
!pip install datasets

**For dataset use Hugging Face model**

*   https://huggingface.co/datasets/osunlp/TravelPlanner




In [ ]:
from groq import Groq
from datasets import load_dataset
import os

client = Groq(api_key="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")

train_ds = load_dataset("osunlp/TravelPlanner", "train")
test_ds = load_dataset("osunlp/TravelPlanner", "test")
validation_ds = load_dataset("osunlp/TravelPlanner", "validation")

print(train_ds['train'][0])

def retrieve_travel_data(dest, days, people_number, local_constraint):
    filtered_data = train_ds['train'].filter(lambda example: dest in example.get('destination', ''))
    return [item['annotated_plan'] for item in filtered_data][:3]

def get_travel_recommendations(org, dest, days, visiting_city_number, date, people_number, local_constraint, query):
    retrieved_info = retrieve_travel_data(dest, days, people_number, local_constraint)
    prompt = f"""
    I am planning a trip from {org} to {dest} for {days} days.
    The trip includes {visiting_city_number} cities, starting on {date}, with {people_number} people.
    The constraints include: {local_constraint}.
    Here is some relevant travel information: {retrieved_info}.
    My query is: {query}.
    Please provide a travel plan that complies with all these requirements.
    """
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    return chat_completion.choices[0].message.content

org = input("Enter the city of origin: ")
dest = input("Enter the destination city: ")
days = int(input("Enter the number of days: "))
visiting_city_number = int(input("Enter the number of cities to visit: "))
date = input("Enter the start date (YYYY-MM-DD): ")
people_number = int(input("Enter the number of people: "))
local_constraint = input("Enter any local constraints (e.g., dietary restrictions, transportation preferences): ")
query = input("Enter your travel query (e.g., what to visit, where to stay): ")

recommendations = get_travel_recommendations(org, dest, days, visiting_city_number, date, people_number, local_constraint, query)

print("\nTravel Recommendations:\n")
print(recommendations)


***For creating Streamlit Environment using Colab ***

In [ ]:
#!pip install streamlit -q
#!wget -q -O - ipv4.icanhazip.com
#streamlit run app.py & npx localtunnel --port 8501

**For Github**  

*   Create a file named as app.py in folder section of colab
Transfer the file from colab to github


*   create a file name as requirements.txt
add the name of dependencies

1.   streamlit
2.   datasets
3.   groq






**Streamlit Code app.py**

In [ ]:

import streamlit as st
from groq import Groq
from datasets import load_dataset

# Initialize the Groq client
client = Groq(api_key="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")

# Load the dataset
@st.cache_resource
def load_data():
    train_ds = load_dataset("osunlp/TravelPlanner", "train")
    return train_ds['train']

train_ds = load_data()

def retrieve_travel_data(dest, days, people_number, local_constraint):
    filtered_data = train_ds.filter(lambda example: dest in example.get('destination', ''))
    return [item['annotated_plan'] for item in filtered_data][:3]

def get_travel_recommendations(org, dest, days, visiting_city_number, date, people_number, local_constraint, query):
    retrieved_info = retrieve_travel_data(dest, days, people_number, local_constraint)
    prompt = f"""
    I am planning a trip from {org} to {dest} for {days} days.
    The trip includes {visiting_city_number} cities, starting on {date}, with {people_number} people.
    The constraints include: {local_constraint}.
    Here is some relevant travel information: {retrieved_info}.
    My query is: {query}.
    Please provide a travel plan that complies with all these requirements.
    """
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    return chat_completion.choices[0].message.content

# Streamlit App
st.title("Travel Planner")

org = st.text_input("Enter the city of origin:")
dest = st.text_input("Enter the destination city:")
days = st.number_input("Enter the number of days:", min_value=1, value=1)
visiting_city_number = st.number_input("Enter the number of cities to visit:", min_value=1, value=1)
date = st.date_input("Enter the start date:")
people_number = st.number_input("Enter the number of people:", min_value=1, value=1)
local_constraint = st.text_input("Enter any local constraints (e.g., dietary restrictions, transportation preferences):")
query = st.text_input("Enter your travel query (e.g., what to visit, where to stay):")

if st.button("Get Recommendations"):
    if not org or not dest or not query:
        st.error("Please fill out all required fields.")
    else:
        recommendations = get_travel_recommendations(org, dest, days, visiting_city_number, date, people_number, local_constraint, query)
        st.subheader("Travel Recommendations:")
        st.write(recommendations)
